In [2]:
import sys
from pathlib import Path

project_root = Path.cwd().parent
sys.path.append(str(project_root))
sys.path.append(str(project_root/'src'))

In [3]:
project_root

PosixPath('/home/jboerma/Documents/tufts-classes/cs150/CS150-FinalProject')

In [4]:
from pathlib import Path

In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F

from torchvision import datasets, transforms
from torch.utils.data import DataLoader

from torch.utils.data import Dataset
import glob

import os, torch

In [6]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [7]:
# DATASET_PATH = Path("/cluster/tufts/liulab/data")

# SAMPLE_DIR = Path("/cluster/tufts/liulab/jboerm01/CS150-FinalProject/samples")
# SAMPLE_DIR.mkdir(exist_ok=True)

# HARVEY_MODEL_DIR = Path("/cluster/tufts/hugheslab/eharve06/CS150-FinalProject/experiments/")

# BOERMA_MODEL_DIR = Path("/cluster/tufts/liulab/jboerm01/CS150-FinalProject/models")
# BOERMA_MODEL_DIR.mkdir(exist_ok=True)

# MNIST_HARVEY_BEST = 'MNIST_HARVEY_BEST'
# MODELS = {
#     MNIST_HARVEY_BEST: HARVEY_MODEL_DIR / "MNIST_UpscalingDDPM2_noise=low_T=5/batch_size=32_epochs=1000_loss=simple_lr=0.01_random_state=42.pth"
# }

In [8]:
DATASET_PATH = Path("./data")

SAMPLE_DIR = Path("../samples")
SAMPLE_DIR.mkdir(exist_ok=True)

HARVEY_MODEL_DIR = Path("../models/experiments")

BOERMA_MODEL_DIR = Path("../models")
BOERMA_MODEL_DIR.mkdir(exist_ok=True)

MNIST_HARVEY_BEST = 'MNIST_HARVEY_BEST'
MNIST_DDPM_1000 = "MNIST_DDPM_1000"
MNIST_DDPM_5 = "MNIST_DDPM_5"
MODELS = {
    MNIST_HARVEY_BEST: HARVEY_MODEL_DIR / "MNIST_UpscalingDDPM2_noise=low_T=5/batch_size=32_epochs=1000_loss=simple_lr=0.01_random_state=42.pth",
    MNIST_DDPM_1000: "/home/jboerma/Documents/tufts-classes/cs150/CS150-FinalProject/models/experiments/MNIST_DDPM_T=1000/batch_size=32_epochs=1000_lr=0.01_random_state=42.pth", 
    MNIST_DDPM_5: "/home/jboerma/Documents/tufts-classes/cs150/CS150-FinalProject/models/experiments/MNIST_DDPM_T=5/batch_size=32_epochs=1000_lr=0.01_random_state=42.pth",
}

## Generate Samples

In [9]:
## Keep all in memory for now
class ChunkedTensorDataset(Dataset):
    def __init__(self, directory: Path):
        self.files = sorted(directory.glob("*.pt"))
        assert len(self.files) > 0, "No .pt files found"

        self.chunks = []
        self.cum_sizes = []

        total = 0
        for f in self.files:
            chunk = torch.load(f, map_location="cpu")
            self.chunks.append(chunk)
            total += chunk.size(0)
            self.cum_sizes.append(total)

    def __len__(self):
        return self.cum_sizes[-1]

    def __getitem__(self, idx):
        if isinstance(idx, slice):
            # Handle slice
            start, stop, step = idx.indices(len(self))
            result = []
            for i in range(start, stop, step):
                result.append(self._get_single(i)[0])
            return torch.stack(result), 0  # dummy label
        else:
            return self._get_single(idx)

    def _get_single(self, idx):
        for i, cs in enumerate(self.cum_sizes):
            if idx < cs:
                prev = 0 if i == 0 else self.cum_sizes[i - 1]
                x = self.chunks[i][idx - prev]

                # --- uint8 → float in [0,1] ---
                if x.dtype == torch.uint8:
                    x = x.float().div_(255)

                return x, 0            
            
        raise IndexError

In [10]:
class ChunkedSampleWriter:
    def __init__(self, out_dir, chunk_size=5000):
        self.out_dir = out_dir
        self.chunk_size = chunk_size
        os.makedirs(out_dir, exist_ok=True)

        self.buffer = []
        self.buffer_count = 0
        self.chunk_idx = 0
        self.total_saved = 0

    def add(self, samples: torch.Tensor):
        """
        samples: Tensor [B, C, H, W] on CPU
        """
        assert samples.device.type == "cpu"
        assert samples.dim() == 4

        self.buffer.append(samples)
        self.buffer_count += samples.size(0)

        if self.buffer_count >= self.chunk_size:
            self._flush()

    def _flush(self):
        chunk = torch.cat(self.buffer, dim=0)

        to_save = chunk[: self.chunk_size]
        remainder = chunk[self.chunk_size :]

        path = os.path.join(
            self.out_dir, f"samples_{self.chunk_idx:04d}.pt"
        )
        torch.save(to_save, path)

        self.total_saved += to_save.size(0)
        self.chunk_idx += 1

        # reset buffer with remainder
        self.buffer = [remainder] if remainder.numel() > 0 else []
        self.buffer_count = remainder.size(0) if remainder.numel() > 0 else 0

    def finalize(self):
        """
        Save remaining samples (< chunk_size)
        """
        if self.buffer_count > 0:
            chunk = torch.cat(self.buffer, dim=0)
            path = os.path.join(
                self.out_dir, f"samples_{self.chunk_idx:04d}.pt"
            )
            torch.save(chunk, path)

            self.total_saved += chunk.size(0)
            self.chunk_idx += 1

            self.buffer = []
            self.buffer_count = 0

In [11]:
writer = ChunkedSampleWriter(
    out_dir="generated/model_A",
    chunk_size=5000,
)

In [12]:
from src.models import UpscalingDDPM
from src.unet import UNet, UpscaleUNet

In [13]:
MODELS[MNIST_HARVEY_BEST].absolute()

PosixPath('/home/jboerma/Documents/tufts-classes/cs150/CS150-FinalProject/notebooks/../models/experiments/MNIST_UpscalingDDPM2_noise=low_T=5/batch_size=32_epochs=1000_loss=simple_lr=0.01_random_state=42.pth')

### Sanity Check

In [14]:
eps_model = UNet(image_channels=1)
gen_model = UpscalingDDPM(eps_model=eps_model, T=5)
best_weights = torch.load(MODELS[MNIST_HARVEY_BEST], map_location="cpu", weights_only=True)
gen_model.load_state_dict(best_weights)

<All keys matched successfully>

In [15]:
def sample_upscalingddpm(model: UpscalingDDPM, batch_size=36, device=device):
    x_T = 0.1309 + 0.2892 * torch.randn(size=(batch_size, 1, 1, 1), device=device)
    #x_T = torch.randn(size=(batch_size, 1, 1, 1))
    x_ts = [x_T]

    ts = torch.arange(start=0, end=model.T)
    alphas = 1.0 - model.betas
    alphas_cumprod = torch.cumprod(alphas, dim=0)
    sqrt_recip_alphas = torch.sqrt(1.0 / alphas)
    sqrt_one_minus_alphas_cumprod = torch.sqrt(1.0 - alphas_cumprod)

    model.eval()

    with torch.no_grad():
        
        for t in reversed(ts):

            x_t = torch.nn.functional.interpolate(x_ts[-1], scale_factor=2, mode="nearest")
            
            logits = model.eps_model(x_t, torch.full((batch_size,), t, device=device))
            mean = sqrt_recip_alphas[t] * (x_t - model.betas[t] / sqrt_one_minus_alphas_cumprod[t] * logits)
            
            if t > 0:
                noise = torch.randn_like(mean)   
                sigma = torch.sqrt(model.betas[t] * (1.0 - alphas_cumprod[t - 1]) / (1.0 - alphas_cumprod[t]))
                x_ts.append(mean + sigma * noise)
                #x_ts.append(mean)
            else:
                x_ts.append(mean)
    
    return list(reversed(x_ts))

In [16]:
from src.models import DDPM

In [17]:
def sample_ddpm(model: DDPM, batch_size=36, device=device):
    batch_size = 36
    x_T = torch.randn(size=(batch_size, 1, 32, 32), device=device)
    x_ts = [x_T]

    ts = torch.arange(start=0, end=model.T)
    alphas = 1.0 - model.betas
    alphas_cumprod = torch.cumprod(alphas, dim=0)
    sqrt_recip_alphas = torch.sqrt(1.0 / alphas)
    sqrt_one_minus_alphas_cumprod = torch.sqrt(1.0 - alphas_cumprod)

    model.eval()
    with torch.no_grad():
        
        for t in reversed(ts):
            
            logits = model.eps_model(x_ts[-1], torch.full((batch_size,), t, device=device))
            mean = sqrt_recip_alphas[t] * (x_ts[-1] - model.betas[t] / sqrt_one_minus_alphas_cumprod[t] * logits)

            if t > 0:
                noise = torch.randn_like(mean)
                sigma = torch.sqrt(model.betas[t] * (1.0 - alphas_cumprod[t - 1]) / (1.0 - alphas_cumprod[t]))
                x_ts.append(mean + sigma * noise)
            else:
                x_ts.append(mean) 

    return list(reversed(x_ts))

In [18]:
ddpm_eps_model = UNet(image_channels=1)
gen_ddpm_model = DDPM(eps_model=eps_model, T=1000)
gen_ddpm_model.load_state_dict(torch.load(MODELS[MNIST_DDPM_1000]))

<All keys matched successfully>

In [19]:
# x_ts = sample_upscalingddpm(gen_model.to(device))

In [20]:
x_ts = sample_ddpm(gen_ddpm_model.to(device), device)

In [21]:
from matplotlib import pyplot as plt

def display_samples(x_t):
    x_t = x_t.cpu()
    ncols, nrows = 6, 6
    fig, axs = plt.subplots(dpi=300, figsize=(4*ncols, 4*nrows), ncols=ncols, nrows=nrows)

    for i in range(6):
        for j in range(6):
        
            axs[i,j].imshow(x_t[(i*6)+j].permute(1, 2, 0), cmap="gray", vmin=x_t[(i*6)+j].min(), vmax=x_t[(i*6)+j].max())
            axs[i,j].axis("off")

    fig.tight_layout()

    plt.show()

In [22]:
display_samples(x_ts[0])

### Generate For Metrics

First, dump samples for each time for the model

In [23]:
from tqdm.auto import tqdm

In [24]:
def get_sample_dir(model_name: str): 
    return SAMPLE_DIR / model_name

In [25]:
def dump_upscaling_samples(model_name: str, image_channels: int, n_samples = 60_000, batch_size=50):
    eps_model = UNet(image_channels=image_channels)
    gen_model = UpscalingDDPM(eps_model=eps_model, T=5)

    model_path = MODELS[model_name]

    best_weights = torch.load(model_path, map_location="cpu", weights_only=True)
    gen_model.load_state_dict(best_weights)

    gen_model.eval()

    gen_model = gen_model.to(device)

    # test batch - list of xt_BCHW by time
    with torch.no_grad():
        samples_ls = sample_upscalingddpm(gen_model, batch_size=1, device=device)
    n_timesteps = len(samples_ls)
    
    # each timestep gets its own folder. samples aligned by timestep
    sample_dir = get_sample_dir(model_name)

    # exit early if already done
    if sample_dir.exists(): 
        return

    sample_dir.mkdir(exist_ok=True)
    writers = [
        ChunkedSampleWriter(sample_dir / f"{t}", 3000) 
        for t in range(n_timesteps)
    ]

    # now save samples
    n_batches_total = n_samples // batch_size + 1
    for _ in tqdm(
        range(n_batches_total),
        desc="Generating samples",
        leave=True,
    ):
        with torch.no_grad():
            samples_ls = sample_upscalingddpm(gen_model, batch_size=batch_size)
        for sample_t, writer_t in zip(samples_ls, writers): 
            # quantize for data storage
            samples_uint8 = torch.round(torch.clamp(sample_t, 0, 1) * 255).to(torch.uint8)
            writer_t.add(samples_uint8.to('cpu'))

    for writer in writers: 
        writer.finalize()


In [26]:
def dump_ddpm_samples(model_name: str, image_channels: int, T: int=5, n_samples = 60_000, batch_size=50):
    ddpm_eps_model = UNet(image_channels=image_channels)
    gen_ddpm_model = DDPM(eps_model=ddpm_eps_model, T=T)
    gen_ddpm_model.load_state_dict(torch.load(MODELS[model_name]))

    model_path = MODELS[model_name]

    best_weights = torch.load(model_path, map_location="cpu", weights_only=True)
    gen_ddpm_model.load_state_dict(best_weights)

    gen_ddpm_model.eval()

    gen_ddpm_model = gen_ddpm_model.to(device)

    # test batch - list of xt_BCHW by time
    with torch.no_grad():
        samples_ls = sample_ddpm(gen_ddpm_model, batch_size=1, device=device)
    n_timesteps = len(samples_ls)
    
    # each timestep gets its own folder. samples aligned by timestep
    sample_dir = get_sample_dir(model_name)

    # exit early if already done
    if sample_dir.exists(): 
        return

    sample_dir.mkdir(exist_ok=True)
    writers = [
        ChunkedSampleWriter(sample_dir / f"{t}", 3000) 
        for t in range(n_timesteps)
    ]

    # now save samples
    n_batches_total = n_samples // batch_size + 1
    for _ in tqdm(
        range(n_batches_total),
        desc="Generating samples",
        leave=True,
    ):
        with torch.no_grad():
            samples_ls = sample_ddpm(gen_ddpm_model, batch_size=batch_size)
        for sample_t, writer_t in zip(samples_ls, writers): 
            # quantize for data storage
            samples_uint8 = torch.round(torch.clamp(sample_t, 0, 1) * 255).to(torch.uint8)
            writer_t.add(samples_uint8.to('cpu'))

    for writer in writers: 
        writer.finalize()


### SAMPLE MODEL

In [27]:
dump_upscaling_samples(MNIST_HARVEY_BEST, 1, 60_000, 100)

In [28]:
dump_ddpm_samples(MNIST_DDPM_5, 1, 5, 60_000, 100)

In [29]:
dump_ddpm_samples(MNIST_DDPM_1000, 1, 1000, 60_000, 1000)

Check!

In [30]:
sample_dir_0 = get_sample_dir(MNIST_HARVEY_BEST) / "0"
dataset = ChunkedTensorDataset(sample_dir_0)
display_samples(dataset[:50][0])

In [31]:
sample_dir_0 = get_sample_dir(MNIST_DDPM_5) / "0"
dataset = ChunkedTensorDataset(sample_dir_0)
display_samples(dataset[:50][0])

In [32]:
sample_dir_0 = get_sample_dir(MNIST_DDPM_1000) / "0"
dataset = ChunkedTensorDataset(sample_dir_0)
display_samples(dataset[:50][0])

## Compute Metrics for Models

### MNIST-FID
The "FID-Statistic" is computed by the deepest layer in the Inception v3
classifier. 

To compute FID between datasets, start by computing all FID-Statistics. We then 
fit a Gaussian to the FID-Statistic vectors for dataset A, and do the same for 
dataset B. The FID is the wasserstein distance between the gaussians, 
which can be computed in closed-form.

#### Training MNIST Classifier

In [33]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class MNISTClassifier32(nn.Module):
    def __init__(self, embedding_dim=128):
        super().__init__()

        # Feature extractor for 32x32 input
        self.conv = nn.Sequential(
            nn.Conv2d(1, 32, 3, stride=1, padding=1),  # 32x32 → 32x32
            nn.ReLU(),
            nn.MaxPool2d(2),                           # 16x16
            nn.Conv2d(32, 64, 3, stride=1, padding=1), # 16x16
            nn.ReLU(),
            nn.MaxPool2d(2),                           # 8x8
        )

        self.fc_embed = nn.Linear(64 * 8 * 8, embedding_dim)
        self.fc_out   = nn.Linear(embedding_dim, 10)

    def forward_features(self, x):
        """
        Returns the penultimate embedding (for FID).
        """
        h = self.conv(x)
        h = h.view(h.size(0), -1)
        h = self.fc_embed(h)
        return h

    def forward(self, x):
        """
        Returns class logits.
        """
        h = self.forward_features(x)
        logits = self.fc_out(F.relu(h))
        return logits


In [34]:
MNIST_CLASSIFER_PATH = BOERMA_MODEL_DIR / "classifer_weights.pt"

In [35]:
import torchvision
from torch.utils.data import DataLoader, random_split

device = "cuda" if torch.cuda.is_available() else "cpu"
mnist_classifier = MNISTClassifier32(embedding_dim=128).to(device)

if MNIST_CLASSIFER_PATH.exists(): 
    weights = torch.load(MNIST_CLASSIFER_PATH)
    mnist_classifier.load_state_dict(weights)
else: 
    transform = torchvision.transforms.Compose([
        torchvision.transforms.Resize(32),
        torchvision.transforms.ToTensor(),
    ])
    
    full_train_ds = torchvision.datasets.MNIST(
        root=DATASET_PATH,
        train=True,
        download=True,
        transform=transform,
    )

    # --- Split into train/validation (e.g., 55k/5k) ---
    val_size = 5000
    train_size = len(full_train_ds) - val_size
    train_ds, val_ds = random_split(full_train_ds, [train_size, val_size])

    train_loader = DataLoader(train_ds, batch_size=128, shuffle=True)
    val_loader   = DataLoader(val_ds, batch_size=256, shuffle=False)

    mnist_classifier = MNISTClassifier32(embedding_dim=128).to(device)
    optimizer = torch.optim.Adam(mnist_classifier.parameters(), lr=1e-3)
    criterion = nn.CrossEntropyLoss()

    num_epochs = 3  # MNIST converges fast
    for epoch in range(num_epochs):
        mnist_classifier.train()
        running_loss = 0.0

        for x, y in train_loader:
            x, y = x.to(device), y.to(device)

            optimizer.zero_grad()
            logits = mnist_classifier(x)
            loss = criterion(logits, y)
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * x.size(0)

        epoch_loss = running_loss / train_size

        # --- Validation ---
        mnist_classifier.eval()
        val_loss = 0.0
        correct = 0
        total = 0
        with torch.no_grad():
            for x_val, y_val in val_loader:
                x_val, y_val = x_val.to(device), y_val.to(device)
                logits = mnist_classifier(x_val)
                loss = criterion(logits, y_val)
                val_loss += loss.item() * x_val.size(0)

                preds = logits.argmax(dim=1)
                correct += (preds == y_val).sum().item()
                total += y_val.size(0)

        val_loss /= val_size
        val_acc = correct / total

        print(f"Epoch {epoch+1}/{num_epochs} | Train Loss: {epoch_loss:.4f} | Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.4f}")

    # Save final weights
    torch.save(mnist_classifier.state_dict(), MNIST_CLASSIFER_PATH)


In [36]:
@torch.no_grad()
def extract_embeddings(model, dataloader, device):
    model.eval()
    embeddings = []

    for x, _ in dataloader:
        x = x.to(device)
        emb = model.forward_features(x)
        embeddings.append(emb.cpu())

    return torch.cat(embeddings, dim=0)

In [37]:
# def get_mnist_embeddings(
#     classifier,
#     device,
#     batch_size=256,
# ):
#     transform = transforms.ToTensor()  # MNIST in [0,1]

#     mnist = datasets.MNIST(
#         root="./data",
#         train=True,
#         download=True,
#         transform=transform,
#     )

#     loader = DataLoader(
#         mnist,
#         batch_size=batch_size,
#         shuffle=False,
#         num_workers=2,
#         pin_memory=True,
#     )

#     embeddings = extract_embeddings(classifier, loader, device)
#     return embeddings

In [38]:
from cleanfid.fid import fid_from_feats, frechet_distance

Let's compute

In [39]:
import torchvision

transform = torchvision.transforms.Compose([
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Resize(size=32),
])
mnist_name_to_set = {
    "MNIST_TRAIN": datasets.MNIST(
        root="./data",
        train=True,
        download=True,
        transform=transform,
    ), 
    "MNIST_TEST": datasets.MNIST(
        root="./data",
        train=False,
        download=True,
        transform=transform,
    ), 
    "UPSCALE_DDPM:": ChunkedTensorDataset(get_sample_dir(MNIST_HARVEY_BEST) / "0"),
    "DDPM_5": ChunkedTensorDataset(get_sample_dir(MNIST_DDPM_5) / "0"), 
    "DDPM_1000": ChunkedTensorDataset(get_sample_dir(MNIST_DDPM_1000) / "0"),
}

In [40]:
import numpy as np

def feats_stats_for_dataset(
    dataset,
    classifier,
    device,
    batch_size=256,
):
    loader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=False,
        num_workers=2,
        pin_memory=True,
    )

    classifier.eval()
    feats = []

    with torch.no_grad():
        for x, _ in loader:
            x = x.to(device, non_blocking=True)
            f = classifier.forward_features(x)
            feats.append(f.cpu())

    feats = torch.cat(feats, dim=0).detach().cpu().numpy()
    
    # from cleanfid
    mu, sig = np.mean(feats, axis=0), np.cov(feats, rowvar=False)

    return mu, sig

In [41]:
@torch.no_grad()
def get_feats_for_dataset(dataset, classifier, device, batch_size=256):
    """
    Returns embeddings (features) for all samples in a dataset.

    Args:
        dataset: PyTorch Dataset returning (x, y)
        classifier: model with forward_features(x)
        device: 'cuda' or 'cpu'
        batch_size: dataloader batch size

    Returns:
        feats: [N, D] torch.FloatTensor on CPU
    """
    loader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=False,
        num_workers=2,
        pin_memory=True,
    )

    classifier.eval()
    feats = []

    for x, _ in tqdm(loader, desc="Extracting features", leave=True):
        x = x.to(device, non_blocking=True)
        f = classifier.forward_features(x)
        feats.append(f.cpu())

    feats = torch.cat(feats, dim=0)
    return feats


In [42]:
def compute_all_stats(
    datasets: dict,
    classifier,
    device,
    batch_size=256,
):
    stats = {}

    for name, ds in datasets.items():
        print(f"Computing stats for {name}...")
        mu, sigma = feats_stats_for_dataset(
            ds, classifier, device, batch_size
        )
        stats[name] = (mu, sigma)

    return stats

In [43]:
mnist_name_to_stats = compute_all_stats(mnist_name_to_set, mnist_classifier, device)

Computing stats for MNIST_TRAIN...
Computing stats for MNIST_TEST...
Computing stats for UPSCALE_DDPM:...
Computing stats for DDPM_5...
Computing stats for DDPM_1000...


In [44]:
def fids_vs_reference(
    stats: dict,
    ref_name: str = "MNIST_TRAIN",
):
    assert ref_name in stats, f"{ref_name} not found"

    mu_ref, sigma_ref = stats[ref_name]

    fids = {}
    for name, (mu, sigma) in stats.items():
        fid = frechet_distance(mu, sigma, mu_ref, sigma_ref)
        fids[name] = fid

    return fids

In [45]:
fids_vs_reference(mnist_name_to_stats, "MNIST_TRAIN")

{'MNIST_TRAIN': np.float64(-2.773958840407431e-10),
 'MNIST_TEST': np.float64(3.1322040240756905),
 'UPSCALE_DDPM:': np.float64(657.7818632376652),
 'DDPM_5': np.float64(1852.455588333078),
 'DDPM_1000': np.float64(16.903518351117782)}

### MNIST-Wass

In [46]:
ddpm_up_5_eps_model = UNet(image_channels=1)
ddpm_up_5_model = UpscalingDDPM(eps_model=eps_model, T=5)
ddpm_up_5_model.load_state_dict(torch.load(MODELS[MNIST_HARVEY_BEST]))

<All keys matched successfully>

In [47]:
ddpm_5_eps_model = UNet(image_channels=1)
ddpm_5_model = DDPM(eps_model=eps_model, T=5)
ddpm_5_model.load_state_dict(torch.load(MODELS[MNIST_DDPM_5]))

<All keys matched successfully>

In [48]:
ddpm_1000_eps_model = UNet(image_channels=1)
ddpm_1000_model = DDPM(eps_model=eps_model, T=1000)
ddpm_1000_model.load_state_dict(torch.load(MODELS[MNIST_DDPM_1000]))

<All keys matched successfully>

In [49]:
mnist_name_to_model = {
    "MNIST_TRAIN": None, 
    "MNIST_TEST": None, 
    "UPSCALE_DDPM:": ddpm_up_5_model,
    "DDPM_5": ddpm_5_model, 
    "DDPM_1000": ddpm_1000_model,
}

First compare alphas. Let's see if we are at similar within-resolution noise scales. 
Critically, we want to choose the best timesteps for DDPM-1000.

In [50]:
for name, model in mnist_name_to_model.items(): 
    if model is None: 
        continue 

    print(name)
    print(model.alpha_bar())

UPSCALE_DDPM:
tensor([0.9999, 0.9948, 0.9848, 0.9700, 0.9506])
DDPM_5
tensor([0.9999, 0.9948, 0.9848, 0.9700, 0.9506])
DDPM_1000
tensor([9.9990e-01, 9.9978e-01, 9.9964e-01, 9.9948e-01, 9.9930e-01, 9.9910e-01,
        9.9888e-01, 9.9864e-01, 9.9838e-01, 9.9811e-01, 9.9781e-01, 9.9749e-01,
        9.9715e-01, 9.9679e-01, 9.9641e-01, 9.9602e-01, 9.9560e-01, 9.9516e-01,
        9.9471e-01, 9.9423e-01, 9.9374e-01, 9.9322e-01, 9.9269e-01, 9.9213e-01,
        9.9156e-01, 9.9097e-01, 9.9035e-01, 9.8972e-01, 9.8907e-01, 9.8840e-01,
        9.8771e-01, 9.8700e-01, 9.8627e-01, 9.8553e-01, 9.8476e-01, 9.8398e-01,
        9.8317e-01, 9.8235e-01, 9.8151e-01, 9.8065e-01, 9.7977e-01, 9.7887e-01,
        9.7795e-01, 9.7702e-01, 9.7606e-01, 9.7509e-01, 9.7410e-01, 9.7309e-01,
        9.7206e-01, 9.7102e-01, 9.6995e-01, 9.6887e-01, 9.6777e-01, 9.6665e-01,
        9.6551e-01, 9.6436e-01, 9.6319e-01, 9.6200e-01, 9.6079e-01, 9.5956e-01,
        9.5832e-01, 9.5706e-01, 9.5578e-01, 9.5449e-01, 9.5318e-01, 9.5

#### ALPHAS COMPARISON
For each alpha level we can compare the distribution of samples generated vs. 
the true noised versions. But comparing model to model, it does not necessarily
make sense to compare two models at the same alpha level (save for the data point x_0), 
since resolution adjustment will change entropy. 

Critically, the DDPM_5 is not getting a fair shot right now, since its x_T 
is definitely not gaussian.

### Sliced Wasserstein At Final Time

In [51]:
device

'cuda'

In [52]:
name_to_feats = {}
for name, ds in mnist_name_to_set.items():
    print(f"Extracting features for {name}...")
    feats = get_feats_for_dataset(ds, mnist_classifier, device, batch_size=1000)
    name_to_feats[name] = feats
    print(f"  Done: {feats.shape[0]} samples, {feats.shape[1]} dims")

Extracting features for MNIST_TRAIN...


Extracting features:   0%|          | 0/60 [00:00<?, ?it/s]

  Done: 60000 samples, 128 dims
Extracting features for MNIST_TEST...


Extracting features:   0%|          | 0/10 [00:00<?, ?it/s]

Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x77340cdbdf80>Exception ignored in: 
<function _MultiProcessingDataLoaderIter.__del__ at 0x77340cdbdf80>Traceback (most recent call last):

  File "/home/jboerma/miniconda3/envs/deep_learning/lib/python3.12/site-packages/torch/utils/data/dataloader.py", line 1663, in __del__
Traceback (most recent call last):
  File "/home/jboerma/miniconda3/envs/deep_learning/lib/python3.12/site-packages/torch/utils/data/dataloader.py", line 1663, in __del__
        self._shutdown_workers()self._shutdown_workers()

  File "/home/jboerma/miniconda3/envs/deep_learning/lib/python3.12/site-packages/torch/utils/data/dataloader.py", line 1646, in _shutdown_workers
  File "/home/jboerma/miniconda3/envs/deep_learning/lib/python3.12/site-packages/torch/utils/data/dataloader.py", line 1646, in _shutdown_workers
        if w.is_alive():if w.is_alive():

              ^^^^^^^^^^^^^^^^^^^^^^
   File "/home/jboerma/miniconda3/envs/deep_learn

...

Extracting features:   0%|          | 0/22 [00:00<?, ?it/s]

Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x77340cdbdf80>
Traceback (most recent call last):
  File "/home/jboerma/miniconda3/envs/deep_learning/lib/python3.12/site-packages/torch/utils/data/dataloader.py", line 1663, in __del__
    self._shutdown_workers()
  File "/home/jboerma/miniconda3/envs/deep_learning/lib/python3.12/site-packages/torch/utils/data/dataloader.py", line 1646, in _shutdown_workers
    if w.is_alive():
       ^^^^^^^^^^^^
  File "/home/jboerma/miniconda3/envs/deep_learning/lib/python3.12/multiprocessing/process.py", line 160, in is_alive
    assert self._parent_pid == os.getpid(), 'can only test a child process'
 Exception ignored in:   <function _MultiProcessingDataLoaderIter.__del__ at 0x77340cdbdf80> 
  Traceback (most recent call last):
   File "/home/jboerma/miniconda3/envs/deep_learning/lib/python3.12/site-packages/torch/utils/data/dataloader.py", line 1663, in __del__
       self._shutdown_workers() 
^  File "/home/jboerma/minic

  Done: 21636 samples, 128 dims
Extracting features for DDPM_1000...


Extracting features:   0%|          | 0/3 [00:00<?, ?it/s]

  Done: 2196 samples, 128 dims


In [53]:
# import torch

# @torch.no_grad()
# def sliced_wasserstein_distance(feats_real, feats_fake, n_projections=100, p=2):
#     """
#     feats_real, feats_fake: [N,D] torch tensors
#     n_projections: number of random directions
#     p: Wasserstein-p distance
#     """
#     N, D = feats_real.shape

#     # Sample random unit directions
#     projections = torch.randn(n_projections, D, device=feats_real.device)
#     projections = F.normalize(projections, dim=1)  # [n_proj, D]

#     swd_total = 0.0
#     for v in projections:
#         # Project features onto v
#         r_proj = feats_real @ v
#         f_proj = feats_fake @ v

#         # Sort projections
#         r_sorted, _ = torch.sort(r_proj)
#         f_sorted, _ = torch.sort(f_proj)

#         # Compute 1D Wasserstein-p distance
#         swd_total += torch.mean(torch.abs(r_sorted - f_sorted)**p).pow(1.0/p)

#     return swd_total / n_projections


In [54]:
import torch

def sliced_wasserstein_distance(feats_real, feats_fake, n_projections=100, p=2, seed=0):
    """
    Compute sliced Wasserstein distance (SWD) between two feature sets,
    handling mismatched sizes by subsampling the larger set.

    Args:
        feats_real: [N_real, D] tensor
        feats_fake: [N_fake, D] tensor
        n_projections: number of random projections
        p: Wasserstein distance power
        seed: random seed for reproducibility

    Returns:
        SWD value (scalar)
    """
    torch.manual_seed(seed)
    N_real, D = feats_real.shape
    N_fake, _ = feats_fake.shape

    # Subsample to match sizes
    if N_real > N_fake:
        idx = torch.randperm(N_real)[:N_fake]
        feats_real = feats_real[idx]
    elif N_fake > N_real:
        idx = torch.randperm(N_fake)[:N_real]
        feats_fake = feats_fake[idx]

    swd_total = 0.0
    for _ in range(n_projections):
        # Random direction
        proj = torch.randn(D, device=feats_real.device)
        proj = proj / proj.norm()
        v_real = feats_real @ proj
        v_fake = feats_fake @ proj

        # Sort projections
        v_real_sorted, _ = torch.sort(v_real)
        v_fake_sorted, _ = torch.sort(v_fake)

        # L^p distance
        swd_total += torch.mean(torch.abs(v_real_sorted - v_fake_sorted) ** p).item()

    swd = (swd_total / n_projections) ** (1 / p)
    return swd


In [55]:
@torch.no_grad()
def nearest_neighbor_distances(feats_real, feats_fake, eps=1e-8):
    """
    feats_real: [N_real, D]
    feats_fake: [N_fake, D]
    
    Returns:
        - mean distance from each fake to nearest real
        - mean distance from each real to nearest fake
    """
    # Compute squared Euclidean distances using broadcasting
    # feats_fake: [Nf,1,D], feats_real: [1,Nr,D] -> [Nf,Nr,D]
    diff = feats_fake[:, None, :] - feats_real[None, :, :]  # [Nf,Nr,D]
    dists = torch.norm(diff, dim=2)  # [Nf, Nr]

    # Fake → Real
    nn_fake_to_real = dists.min(dim=1)[0].mean().item()
    # Real → Fake
    nn_real_to_fake = dists.min(dim=0)[0].mean().item()

    return nn_fake_to_real, nn_real_to_fake

In [56]:
@torch.no_grad()
def nearest_neighbor_distances_streamed(feats_real, feats_fake, batch_size=256):
    """
    Memory-efficient nearest neighbor distances.
    
    feats_real: [N_real, D] torch tensor
    feats_fake: [N_fake, D] torch tensor
    batch_size: process fake embeddings in chunks
    
    Returns:
        nn_fake_to_real, nn_real_to_fake
    """
    N_real = feats_real.size(0)
    N_fake = feats_fake.size(0)

    same_tensor = feats_real.data_ptr() == feats_fake.data_ptr()

    # --- Fake → Real ---
    nn_fake_to_real_sum = 0.0
    for start in tqdm(range(0, N_fake, batch_size), desc="Fake→Real"):
        end = min(start + batch_size, N_fake)
        f_batch = feats_fake[start:end]  # [B, D]

        dists = torch.cdist(f_batch, feats_real)  # [B, N_real]

        # Mask self-distances if it's the same tensor
        if same_tensor:
            for i, idx in enumerate(range(start, end)):
                dists[i, idx] = float('inf')

        nn_fake_to_real_sum += dists.min(dim=1)[0].sum().item()

    nn_fake_to_real = nn_fake_to_real_sum / N_fake

    # --- Real → Fake ---
    nn_real_to_fake_sum = 0.0
    for start in tqdm(range(0, N_real, batch_size), desc="Real→Fake"):
        end = min(start + batch_size, N_real)
        r_batch = feats_real[start:end]

        dists = torch.cdist(r_batch, feats_fake)

        # Mask self-distances if it's the same tensor
        if same_tensor:
            for i, idx in enumerate(range(start, end)):
                dists[i, idx] = float('inf')

        nn_real_to_fake_sum += dists.min(dim=1)[0].sum().item()

    nn_real_to_fake = nn_real_to_fake_sum / N_real

    return nn_fake_to_real, nn_real_to_fake

In [57]:
name_to_feats

{'MNIST_TRAIN': tensor([[  4.8274, -21.4609,  -4.3670,  ...,  -3.5085,  -6.6111,  -4.6885],
         [ -6.7202,   3.9364,   1.5276,  ...,  -5.2186,   8.8348,   6.0143],
         [  2.0932,   5.4570,   4.5937,  ...,  -4.6242,  -4.9732,   0.7994],
         ...,
         [  5.3182, -16.2919,  -4.3226,  ...,  -2.4400,  -3.9770,  -6.3974],
         [ -2.2805,  -3.9133,   6.6577,  ...,  -4.6843,   5.6587,   5.3933],
         [  0.9533,   0.0579,  -6.9613,  ...,  -3.0545,  -1.3468,  -5.4660]]),
 'MNIST_TEST': tensor([[  4.9856,  -1.0631,  -0.9078,  ...,  -3.7625,  -1.4865,   1.8103],
         [ -8.8950,  -8.0520,  -1.2445,  ...,  -6.0942,   2.6047,   3.3353],
         [ -2.2408,   8.3092,   3.4004,  ...,  -2.1498,   1.3332,   0.0595],
         ...,
         [  3.6020,   2.6009,   5.5641,  ...,  -4.6404,   0.5484,  -0.2507],
         [  3.9052,  -7.0186,  -0.2372,  ...,  -3.5523,  -1.3269,  -4.2361],
         [ -5.8871, -10.7847,   6.1563,  ...,  -6.9138,   8.6733,   6.0489]]),
 'UPSCALE_DDPM:

In [58]:
def nnd_vs_reference(
    feats: dict,
    ref_name: str = "MNIST_TRAIN",
):
    assert ref_name in feats, f"{ref_name} not found"

    feats_ref = feats[ref_name]

    dists_ftr_rtf = {}
    for name, feats in feats.items():
        print(name)
        nn_fake_to_real, nn_real_to_fake = nearest_neighbor_distances_streamed(feats_ref, feats)
        dists_ftr_rtf[name] = nn_fake_to_real, nn_real_to_fake

    return dists_ftr_rtf

In [59]:
nnd_to_train = nnd_vs_reference(name_to_feats, "MNIST_TRAIN")

MNIST_TRAIN


Fake→Real:   0%|          | 0/235 [00:00<?, ?it/s]

Real→Fake:   0%|          | 0/235 [00:00<?, ?it/s]

MNIST_TEST


Fake→Real:   0%|          | 0/40 [00:00<?, ?it/s]

...

Fake→Real:   0%|          | 0/85 [00:00<?, ?it/s]

Real→Fake:   0%|          | 0/235 [00:00<?, ?it/s]

DDPM_1000


Fake→Real:   0%|          | 0/9 [00:00<?, ?it/s]

Real→Fake:   0%|          | 0/235 [00:00<?, ?it/s]

In [60]:
nnd_to_train
# model to train, train to real

{'MNIST_TRAIN': (10.789213080851237, 10.789213080851237),
 'MNIST_TEST': (11.001096913146972, 13.231163177490235),
 'UPSCALE_DDPM:': (11.814785842768563, 18.238093241373697),
 'DDPM_5': (23.22750311167585, 40.28732104899088),
 'DDPM_1000': (10.671094161348048, 15.523430285644531)}

In [61]:
def swd_vs_reference(feats_dict, ref_name="MNIST_TRAIN", n_projections=100, p=2):
    """
    Compute sliced Wasserstein distance of all datasets versus a reference.

    Args:
        feats_dict: dict[name -> feats tensor [N,D]]
        ref_name: name of reference dataset
        n_projections: number of random projections
        p: Wasserstein distance power

    Returns:
        dict[name -> SWD value]
    """
    assert ref_name in feats_dict, f"{ref_name} not found"
    feats_ref = feats_dict[ref_name]

    swd_results = {}
    for name, feats_cur in feats_dict.items():
        print(f"Computing SWD for {name} vs {ref_name}...")
        swd_val = sliced_wasserstein_distance(feats_ref, feats_cur, n_projections=n_projections, p=p)
        swd_results[name] = swd_val

    return swd_results


In [62]:
swd_results = swd_vs_reference(name_to_feats, ref_name="MNIST_TRAIN", n_projections=1000)

for name, swd_val in swd_results.items():
    print(f"{name}: SWD={swd_val:.4f}")

Computing SWD for MNIST_TRAIN vs MNIST_TRAIN...
Computing SWD for MNIST_TEST vs MNIST_TRAIN...
Computing SWD for UPSCALE_DDPM: vs MNIST_TRAIN...
Computing SWD for DDPM_5 vs MNIST_TRAIN...
Computing SWD for DDPM_1000 vs MNIST_TRAIN...
MNIST_TRAIN: SWD=0.0000
MNIST_TEST: SWD=0.1345
UPSCALE_DDPM:: SWD=1.8402
DDPM_5: SWD=3.7123
DDPM_1000: SWD=0.3979


### MNIST-MMD

In [63]:
def compute_mmd(feats_real, feats_fake, kernel_mul=2.0, kernel_num=5, fix_sigma=None):
    """
    Compute Maximum Mean Discrepancy (MMD) between two sets of features using RBF kernels.

    Args:
        feats_real: [N_real, D] torch tensor
        feats_fake: [N_fake, D] torch tensor
        kernel_mul: scaling factor for bandwidth
        kernel_num: number of bandwidths to use
        fix_sigma: if provided, uses this as the RBF sigma

    Returns:
        mmd: scalar MMD value
    """
    N_real = feats_real.size(0)
    N_fake = feats_fake.size(0)
    
    # Concatenate features
    feats = torch.cat([feats_real, feats_fake], dim=0)  # [N_real+N_fake, D]
    
    # Compute pairwise squared distances
    xx = torch.matmul(feats, feats.t())
    x2 = torch.sum(feats ** 2, dim=1, keepdim=True)
    pairwise_distances = x2 + x2.t() - 2 * xx  # [N_total, N_total]

    # Compute bandwidths
    if fix_sigma:
        bandwidth_list = [fix_sigma]
    else:
        # Median heuristic
        median_dist = torch.median(pairwise_distances)
        base_bandwidth = median_dist / (kernel_mul ** (kernel_num // 2))
        bandwidth_list = [base_bandwidth * (kernel_mul ** i) for i in range(kernel_num)]

    # Compute kernels
    kernels = [torch.exp(-pairwise_distances / (2 * bw)) for bw in bandwidth_list]
    K = sum(kernels)  # [N_total, N_total]

    # Split kernel matrix
    K_xx = K[:N_real, :N_real]
    K_yy = K[N_real:, N_real:]
    K_xy = K[:N_real, N_real:]
    K_yx = K_xy.t()

    # MMD statistic
    mmd = K_xx.mean() + K_yy.mean() - K_xy.mean() - K_yx.mean()
    return mmd.item()


In [64]:
import torch
from tqdm.notebook import tqdm

@torch.no_grad()
def compute_mmd_streamed(feats_real, feats_fake, kernel_mul=2.0, kernel_num=5, fix_sigma=None, batch_size=512):
    """
    Memory-efficient MMD using block-wise computation.

    Args:
        feats_real: [N_real, D]
        feats_fake: [N_fake, D]
        kernel_mul: RBF scaling factor
        kernel_num: number of RBF kernels
        fix_sigma: fixed sigma if desired
        batch_size: chunk size for streaming

    Returns:
        scalar MMD value
    """
    N_real = feats_real.size(0)
    N_fake = feats_fake.size(0)

    # Compute sigma / bandwidths
    with torch.no_grad():
        if fix_sigma:
            bandwidth_list = [fix_sigma]
        else:
            # Median heuristic on a small subset
            sample_r = feats_real[torch.randperm(N_real)[:min(1024,N_real)]]
            sample_f = feats_fake[torch.randperm(N_fake)[:min(1024,N_fake)]]
            sample = torch.cat([sample_r, sample_f], dim=0)
            pdists = torch.cdist(sample, sample, p=2) ** 2
            median_dist = torch.median(pdists)
            base_bandwidth = median_dist / (kernel_mul ** (kernel_num // 2))
            bandwidth_list = [base_bandwidth * (kernel_mul ** i) for i in range(kernel_num)]

    K_xx_sum, K_yy_sum, K_xy_sum = 0.0, 0.0, 0.0
    count_xx, count_yy, count_xy = 0, 0, 0

    # --- K_xx ---
    for i in tqdm(range(0, N_real, batch_size), desc="K_xx"):
        end_i = min(i + batch_size, N_real)
        block_i = feats_real[i:end_i]
        for j in range(0, N_real, batch_size):
            end_j = min(j + batch_size, N_real)
            block_j = feats_real[j:end_j]

            pdist = torch.cdist(block_i, block_j) ** 2
            K_block = sum(torch.exp(-pdist / (2*bw)) for bw in bandwidth_list)

            # Exclude diagonal if i==j to avoid self-pairs?
            if i == j:
                mask = torch.ones_like(K_block, dtype=torch.bool)
                mask.fill_diagonal_(False)
                K_block = K_block[mask].view(K_block.size(0), -1)

            K_xx_sum += K_block.sum().item()
            count_xx += K_block.numel()

    # --- K_yy ---
    for i in tqdm(range(0, N_fake, batch_size), desc="K_yy"):
        end_i = min(i + batch_size, N_fake)
        block_i = feats_fake[i:end_i]
        for j in range(0, N_fake, batch_size):
            end_j = min(j + batch_size, N_fake)
            block_j = feats_fake[j:end_j]

            pdist = torch.cdist(block_i, block_j) ** 2
            K_block = sum(torch.exp(-pdist / (2*bw)) for bw in bandwidth_list)

            if i == j:
                mask = torch.ones_like(K_block, dtype=torch.bool)
                mask.fill_diagonal_(False)
                K_block = K_block[mask].view(K_block.size(0), -1)

            K_yy_sum += K_block.sum().item()
            count_yy += K_block.numel()

    # --- K_xy ---
    for i in tqdm(range(0, N_real, batch_size), desc="K_xy"):
        end_i = min(i + batch_size, N_real)
        block_i = feats_real[i:end_i]
        for j in range(0, N_fake, batch_size):
            end_j = min(j + batch_size, N_fake)
            block_j = feats_fake[j:end_j]

            pdist = torch.cdist(block_i, block_j) ** 2
            K_block = sum(torch.exp(-pdist / (2*bw)) for bw in bandwidth_list)

            K_xy_sum += K_block.sum().item()
            count_xy += K_block.numel()

    mmd = (K_xx_sum / count_xx) + (K_yy_sum / count_yy) - 2 * (K_xy_sum / count_xy)
    return mmd


In [65]:
def mmd_vs_reference(feats_dict, ref_name="MNIST_TRAIN", kernel_mul=2.0, kernel_num=5, fix_sigma=None):
    """
    Compute MMD of all datasets versus a reference dataset.

    Args:
        feats_dict: dict[name -> feats tensor [N,D]]
        ref_name: name of reference dataset
        kernel_mul: scaling factor for RBF kernel
        kernel_num: number of bandwidths to use
        fix_sigma: if provided, fixes the RBF sigma

    Returns:
        dict[name -> MMD value]
    """
    assert ref_name in feats_dict, f"{ref_name} not found"
    feats_ref = feats_dict[ref_name]

    mmd_results = {}
    for name, feats_cur in feats_dict.items():
        print(f"Computing MMD for {name} vs {ref_name}...")
        mmd_val = compute_mmd_streamed(feats_ref, feats_cur, kernel_mul=kernel_mul, kernel_num=kernel_num, fix_sigma=fix_sigma)
        mmd_results[name] = mmd_val

    return mmd_results


In [66]:
mmd_results = mmd_vs_reference(name_to_feats, ref_name="MNIST_TRAIN")

for name, val in mmd_results.items():
    print(f"{name}: MMD={val:.4f}")

Computing MMD for MNIST_TRAIN vs MNIST_TRAIN...


K_xx:   0%|          | 0/118 [00:00<?, ?it/s]

K_yy:   0%|          | 0/118 [00:00<?, ?it/s]

K_xy:   0%|          | 0/118 [00:00<?, ?it/s]

Computing MMD for MNIST_TEST vs MNIST_TRAIN...


...

Computing MMD for DDPM_1000 vs MNIST_TRAIN...


K_xx:   0%|          | 0/118 [00:00<?, ?it/s]

K_yy:   0%|          | 0/5 [00:00<?, ?it/s]

K_xy:   0%|          | 0/118 [00:00<?, ?it/s]

MNIST_TRAIN: MMD=-0.0001
MNIST_TEST: MMD=0.0008
UPSCALE_DDPM:: MMD=0.3286
DDPM_5: MMD=1.5189
DDPM_1000: MMD=0.0048


### CIFAR-10-FID